In [13]:
import pandas as pd
import numpy as np
import torch


# Ajustar y transformar los datos
def Normalizar(escalas,dia,mes,year):

    min_parada,max_parada=0,3
    day_min,day_max=1,31
    mount_min,mount_max=2,4
    year_min,year_max=2022.0,2022
    #Normalizacion
    escalas=(escalas-min_parada)/(max_parada-min_parada)
    dia=(dia-day_min)/(day_max-day_min)
    mes=(mes-mount_min)/(mount_max-mount_min)
    try:
        year= (year-year_min)/(year_max-year_min)
    except:
        year=0


    return [escalas,dia,mes,year]

def preparacion_datos(aerolinea,source,destino):
       aero="Air_"+aerolinea
       source="Source_"+source
       destino="Destination_"+destino
       aerolines={'Air_Aeroflot':0, 'Air_Air Canada':1, 'Air_Air France':2,
       'Air_American Airlines':3, 'Air_British Airways':4, 'Air_Delta':5,
       'Air_Emirates':6, 'Air_Finnair':7, 'Air_Finnair, American Airlines':8,
       'Air_KLM':9, 'Air_LOT':10, 'Air_Lufthansa':11, 'Air_Lufthansa, Egypt Air':12,
       'Air_Multiple Airlines':13, 'Air_Qatar Airways':14, 'Air_SAUDIA':15, 'Air_SWISS':16,
       'Air_TAP AIR PORTUGAL':17, 'Air_Turkish Airlines':18, 'Air_United Airlines':19}
       sources={'Source_NYC':0, 'Source_PAR':1, 'Source_RUH':2, 'Source_SVO':3}
       Destinos={'Destination_NYC':0, 'Destination_PAR':1, 'Destination_RUH':2,'Destination_SVO':3}
       valor_aero=aerolines[aero]
       valor_source=sources[source]
       valor_destino=Destinos[destino]
       lista_aero=np.zeros(20)
       lista_source=np.zeros(4)
       lista_destino=np.zeros(4)
       lista_aero[valor_aero] = 1
       lista_source[valor_source]= 1
       lista_destino[valor_destino]= 1
       return list(lista_aero)+list(lista_source)+list(lista_destino)
     

In [14]:
from datetime import datetime
from datetime import timedelta

def Fecha(Fechas,cant_fechas):
    fecha = datetime.strptime(Fechas, "%Y-%m-%d")
    Fechas=[fecha+timedelta(days=i) for i in range(0,cant_fechas)]
    return Fechas

In [21]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import torch

# Crear el objeto MinMaxScaler
scaler = MinMaxScaler()
Data=pd.read_csv("./modelo/Datos_Transformados.csv")
# Ajustar y transformar los datos
Data['Total stops'] = scaler.fit_transform(Data[['Total stops']])
Data['Departure_Day'] = scaler.fit_transform(Data[['Departure_Day']])
Data['Departure_Month'] = scaler.fit_transform(Data[['Departure_Month']])
Data['Departure_Year'] = scaler.fit_transform(Data[['Departure_Year']])
Data['Price'] = scaler.fit_transform(Data[['Price']])

model = torch.jit.load('./modelo/modelo_precios_boleto.pt')
model.eval()

RecursiveScriptModule(
  original_name=RedPrecios
  (linear_relu_stack): RecursiveScriptModule(
    original_name=Sequential
    (0): RecursiveScriptModule(original_name=Linear)
    (1): RecursiveScriptModule(original_name=ReLU)
    (2): RecursiveScriptModule(original_name=Linear)
    (3): RecursiveScriptModule(original_name=ReLU)
    (4): RecursiveScriptModule(original_name=Linear)
  )
)

In [15]:

def Datos_Pronostico(aerolinea,source,Destino,escalas,fecha,cant_fechas):
     Datos_base=preparacion_datos(aerolinea,source,Destino)
     Dates=Fecha(fecha,cant_fechas)
     Datos_completos=[]
     for date in Dates:
          Datos_Normalizados=Normalizar(escalas,date.day,date.month,date.year)
          Datos_completos.append(Datos_base+Datos_Normalizados)
     return Datos_completos
     

In [17]:
aerolinea='American Airlines'
source='NYC'
Destino='SVO'
resultado=Datos_Pronostico(aerolinea,source,Destino,1,"2023-06-21",4)

In [19]:
for i in resultado:
    print(i)

[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.3333333333333333, 0.6666666666666666, 2.0, 0]
[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.3333333333333333, 0.7, 2.0, 0]
[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.3333333333333333, 0.7333333333333333, 2.0, 0]
[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.3333333333333333, 0.7666666666666667, 2.0, 0]


In [45]:
def Predic(lista_datos):
    lista_datos = [[float(valor) for valor in fila] for fila in lista_datos]
    device = torch.device('cpu')
    S = torch.tensor(lista_datos, device=device)
    list_predic=model(S).detach().numpy()

    return scaler.inverse_transform(list_predic)

In [46]:
ss=Predic(resultado)

In [47]:
ss

array([[1011.22284],
       [1017.7316 ],
       [1024.2407 ],
       [1030.7495 ]], dtype=float32)

In [44]:
[scaler.inverse_transform(ss)]

[array([[1011.22284],
        [1017.7316 ],
        [1024.2407 ],
        [1030.7495 ]], dtype=float32)]

In [33]:
ss[0][0]

0.06629075